In [1]:
import sys
sys.path.append('../../30_data_tools/')
import sqlite3
import pandas as pd
from PIL import ImageFont, ImageDraw
from tqdm.auto import tqdm

In [2]:
from helper import load_dotenv, get_pdf_page_processing_status
from file_interaction import blob_exists, open_img, get_generic_image_filepath, get_data_files

In [3]:
from file_interaction import get_blobs

In [4]:
import re

In [5]:
def generate_marker_image( pdf_row ):
    filepath = get_generic_image_filepath(
        pdf_row.pdf_filename,
        pdf_row.job,
        pdf_row.method,
        pdf_row.idx
    )

    if filepath:
        img = open_img( filepath ).convert('RGB')

        # Masken laden
        c = con.cursor()
        sql_string = f'''
            SELECT bbox,ssim FROM mask
            WHERE 
                pdf_filename='{ pdf_row.pdf_filename }' AND
                job='{ pdf_row.job }' AND
                "type"='{ pdf_row["type"] }' AND
                variant_name='{ pdf_row.variant_name }' AND
                method='{ pdf_row.method }' AND
                idx='{ pdf_row.idx }'
        '''
        c.execute(sql_string)
        masks = c.fetchall()
        c.close

        for bbox_string,ssim_value in masks:
            bbox = [int(val) for val in bbox_string.split(';')]
            color = (255,0,0,255) if ssim_value < 0.95 else (0,0,255,255)
            font = ImageFont.load_default(size=50)      
            d = ImageDraw.Draw(img, 'RGBA')
            d.rectangle(
                [bbox[0],bbox[1],bbox[0]+bbox[2],bbox[1]+bbox[3]],
                outline=color,
                width=10
            )
            d.text((bbox[0],bbox[1]), str(ssim_value), fill=color, font=font)

        out_path = f'./10_out/{ pdf_row.job }.{ pdf_row.pdf_filename }.{ pdf_row.method }.{ pdf_row.idx }.marker.jpg'
        
        img.save( out_path, progressive=True, dpi=( dotenv['LOFI_DPI'], dotenv['LOFI_DPI'] ) )

In [6]:
dotenv = load_dotenv()
con = sqlite3.connect( dotenv['DB_PATH'] )

In [7]:
pages = pd.read_sql(
    '''
        SELECT * FROM generic_image
        WHERE job='24-03-05-01_randomTrainPages'
    ''',
    con,
    parse_dates=['timestamp']
)

In [8]:
pages

,pdf_filename,job,type,variant_name,method,idx,timestamp
0,617673.tg39_p1200_U01_800-500.00TIT-057SA.49DE...,24-03-05-01_randomTrainPages,4c,halftone600dpi,soft_light,1,2024-03-09 10:52:44.319136
1,617277.tg46_p1209_U4U1_TCD_KA01_PWS_V1_KW03_Ac...,24-03-05-01_randomTrainPages,4c,halftone600dpi,soft_light,1,2024-03-09 11:47:39.021702
2,121287.0040_0049_gaymann.p5,24-03-05-01_randomTrainPages,4c,halftone600dpi,soft_light,1,2024-03-09 11:56:02.093878
3,150232.IO43_gmg-konv43_9783060662517 Inhalt_S0...,24-03-05-01_randomTrainPages,4c,halftone600dpi,soft_light,1,2024-03-09 11:59:03.398673
4,509403.08_HL_2305.p1,24-03-05-01_randomTrainPages,4c,halftone600dpi,soft_light,1,2024-03-09 11:59:52.198640
...,...,...,...,...,...,...,...
115,148965.tg39_DO01_3-12-735361_330-379_K10.p4,24-03-05-01_randomTrainPages,4c,halftone600dpi,soft_light,1,2024-03-12 20:07:17.413644
116,148132.tg39_005-740_hansgrohe_GB_PrintNEU.p270,24-03-05-01_randomTrainPages,4c,halftone600dpi,soft_light,1,2024-03-12 20:10:16.277225
117,145920.g39_003-142_Booklet_Addensare_03_04_202...,24-03-05-01_randomTrainPages,4c,halftone600dpi,soft_light,1,2024-03-12 20:14:34.211355
118,141143.g47_001-160_0867-1_Weidenweber_Kraeuter...,24-03-05-01_randomTrainPages,4c,halftone600dpi,soft_light,1,2024-03-12 20:17:11.953578


In [9]:
for i in tqdm(range(pages.shape[0])):
    row = pages.iloc[i]

    generate_marker_image( row )

  0%|          | 0/120 [00:00<?, ?it/s]

/Users/frederic.birwe/Library/Python/3.10/lib/python/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (124399302 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/Users/frederic.birwe/Library/Python/3.10/lib/python/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (120330965 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/Users/frederic.birwe/Library/Python/3.10/lib/python/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (94735298 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
